In [141]:
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langchain.messages import HumanMessage, AIMessage, ToolMessage
from langchain.tools import tool, ToolRuntime
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware
from langchain.agents import AgentState
from langgraph.types import Command

from pydantic import BaseModel, Field
import yfinance as yf

import os
from dotenv import load_dotenv
import requests
from pprint import pprint

load_dotenv()

True

## Langchain Tracing

In [13]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Stock Fundamentals"

In [14]:
chat_model = init_chat_model(model = "gemma3:1b", model_provider="ollama", temperature=0.7) # initializing a model

In [15]:
res = chat_model.invoke([HumanMessage(content="Hello, how are you?")])

In [16]:
res

AIMessage(content='Hello there! I’m doing well, thank you for asking. As an AI, I don’t experience feelings in the same way humans do, but I’m functioning perfectly and ready to help you with whatever you need. 😊 \n\nHow about you? How’s your day going so far?', additional_kwargs={}, response_metadata={'model': 'gemma3:1b', 'created_at': '2026-01-05T08:35:09.9051045Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1291615000, 'load_duration': 527532900, 'prompt_eval_count': 15, 'prompt_eval_duration': 72321000, 'eval_count': 64, 'eval_duration': 634647400, 'logprobs': None, 'model_name': 'gemma3:1b', 'model_provider': 'ollama'}, id='lc_run--019b8d4b-b8c0-7e30-be14-f2055ddf0314-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 15, 'output_tokens': 64, 'total_tokens': 79})

# Tools

In [124]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
query = "What is the stock ticker symbol for Apple Inc.?, provide only the ticker symbol."
result = search.run(query)
pprint(result)

('In Europe, Apple Inc . is primarily traded on the NASDAQ stock exchange '
 'under the ticker symbol "AAPL." However, if you\'re looking for its '
 'representation on European stock exchanges, such as the Frankfurt Stock '
 'Exchange, it may be listed under the ticker "APC." View today\'s NVIDIA '
 'Corporation stock price and latest NVDA news and analysis. Create real-time '
 'notifications to follow any changes in the live stock price. A stock ticker '
 "symbol is a unique abbreviation used to identify a publicly traded company's "
 "stock on the stock exchange. It's typically a short sequence of letters, "
 'sometimes including numbers. Apple calls it "Time Machine" but it is '
 'actually ZFS at the operating system layer, which is a highly-competent '
 'snapshot, backup and restore mechanism. Use it regularly and make damn sure '
 'the backups are rotated off-site. It will accept a stock ticker symbol as a '
 'parameter and return a mock stock price.This endpoint should accept a G

In [125]:
class ticker_symbol(BaseModel):
    ticker: str = Field(..., description="The stock ticker symbol of the company to analyze.")

In [184]:
@tool
def get_ticker_symbol(company_name: str) -> str:
    """Get the stock ticker symbol for a given company name."""
    search = DuckDuckGoSearchRun()
    query = f"What is the stock ticker symbol for {company_name}?, provide only the ticker symbol."
    result = search.run(query)
    model_vinod = init_chat_model(model="qwen3-vl:2b", model_provider="ollama", temperature=0)
    agent_vinod = create_agent(
        model=model_vinod,
        system_prompt="You are an assistant that finds the ticker symbol of a company based on its name and an article snippet. You must return only the ticker symbol.",
        response_format=ticker_symbol
    )
    response = agent_vinod.invoke([HumanMessage(content=f"Here is the article snippet: {result}. What is the ticker symbol for {company_name}?")])
    return response['structured_response'].ticker
    

In [ ]:
model_vinod = init_chat_model(model="qwen3-vl:2b", model_provider="ollama", temperature=0)
agent = create_agent(
    model=model_vinod,
    system_prompt="You are an assistant that finds the ticker symbol of a company based on its name and an article snippet. You must return only the ticker symbol.",
    response_format=ticker_symbol
)

In [183]:
company_name = "Apple Inc."
response = agent_vinod.invoke({"messages":[HumanMessage(content=f"Here is the article snippet: {result}. What is the ticker symbol for {company_name}?")]})

ResponseError: registry.ollama.ai/library/gemma3:1b does not support tools (status code: 400)

In [134]:
response['structured_response'].ticker

'AAPL'

In [17]:
@tool
def get_income_statement(ticker: str) -> str:
    """
    Useful for analyzing a company's profitability over time.
    Returns the Income Statement, including Revenue, Cost of Revenue, Gross Profit, Operating Expenses, and Net Income.
    Use this to answer questions about: Sales growth, margins, or earnings.
    """
    try:
        stock = yf.Ticker(ticker)

        income_statement = stock.financials

        finances = "Income Statement Table:\n" + income_statement.to_string()

        return finances
    except Exception as e:
        return f"Error fetching income statement for {ticker}: {str(e)}"


In [69]:
res = get_income_statement.invoke({"ticker": "AAPL"})

In [70]:
print(res)

Income Statement Table:
                                                              2025-09-30    2024-09-30    2023-09-30    2022-09-30    2021-09-30
Tax Effect Of Unusual Items                                 0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00           NaN
Tax Rate For Calcs                                          1.560000e-01  2.410000e-01  1.470000e-01  1.620000e-01           NaN
Normalized EBITDA                                           1.447480e+11  1.346610e+11  1.258200e+11  1.305410e+11           NaN
Net Income From Continuing Operation Net Minority Interest  1.120100e+11  9.373600e+10  9.699500e+10  9.980300e+10           NaN
Reconciled Depreciation                                     1.169800e+10  1.144500e+10  1.151900e+10  1.110400e+10           NaN
Reconciled Cost Of Revenue                                  2.209600e+11  2.103520e+11  2.141370e+11  2.235460e+11           NaN
EBITDA                                                      1.447480e+11 

In [22]:
@tool
def get_balance_sheet(ticker: str):
    """
    Useful for analyzing a company's financial health at a specific point in time.
    Returns the Balance Sheet, including Assets (Cash, Inventory), Liabilities (Debt, Payables), and Shareholder Equity.
    Use this to answer questions about: Debt levels, cash position, or book value.
    """
    try:
        stock = yf.Ticker(ticker)

        balance_sheet = stock.balance_sheet

        finances = "Balance Sheet Table:\n" + balance_sheet.to_string()

        return finances
    except Exception as e:
        return f"Error fetching balance sheet for {ticker}: {str(e)}"

In [23]:
res = get_balance_sheet.invoke({"ticker": "AAPL"})

In [25]:
print(res)

Balance Sheet Table:
                                                       2025-09-30    2024-09-30    2023-09-30    2022-09-30    2021-09-30
Treasury Shares Number                                        NaN           NaN  0.000000e+00           NaN           NaN
Ordinary Shares Number                               1.477326e+10  1.511679e+10  1.555006e+10  1.594342e+10           NaN
Share Issued                                         1.477326e+10  1.511679e+10  1.555006e+10  1.594342e+10           NaN
Net Debt                                             6.272300e+10  7.668600e+10  8.112300e+10  9.642300e+10           NaN
Total Debt                                           9.865700e+10  1.066290e+11  1.110880e+11  1.324800e+11           NaN
Tangible Book Value                                  7.373300e+10  5.695000e+10  6.214600e+10  5.067200e+10           NaN
Invested Capital                                     1.723900e+11  1.635790e+11  1.732340e+11  1.707410e+11           NaN
Wor

In [27]:
@tool
def get_cash_flows(ticker: str):
    """
    Useful for analyzing how cash enters and leaves the company.
    Returns the Cash Flow Statement, including Operating Cash Flow, Investing (Capex), and Financing activities.
    Use this to answer questions about: Free Cash Flow, burn rate, or capital expenditures.
    """
    try:
        stock = yf.Ticker(ticker)

        cash_flow = stock.cashflow

        finances = "Cash Flow Table:\n" + cash_flow.to_string()

        return finances
    except Exception as e:
        return f"Error fetching cash flows for {ticker}: {str(e)}"

In [28]:
res = get_cash_flows.invoke({"ticker": "AAPL"})

In [30]:
print(res)

Cash Flow Table:
                                                  2025-09-30    2024-09-30    2023-09-30    2022-09-30    2021-09-30
Free Cash Flow                                  9.876700e+10  1.088070e+11  9.958400e+10  1.114430e+11           NaN
Repurchase Of Capital Stock                    -9.071100e+10 -9.494900e+10 -7.755000e+10 -8.940200e+10           NaN
Repayment Of Debt                              -1.093200e+10 -9.958000e+09 -1.115100e+10 -9.543000e+09           NaN
Issuance Of Debt                                4.481000e+09  0.000000e+00  5.228000e+09  5.465000e+09           NaN
Issuance Of Capital Stock                                NaN           NaN           NaN           NaN  1.105000e+09
Capital Expenditure                            -1.271500e+10 -9.447000e+09 -1.095900e+10 -1.070800e+10           NaN
Interest Paid Supplemental Data                          NaN           NaN  3.803000e+09  2.865000e+09  2.687000e+09
Income Tax Paid Supplemental Data              

In [31]:
@tool
def get_key_metrics_info(ticker: str):
    """
    Useful for getting a current snapshot of the stock's valuation and market data.
    Returns a dictionary containing: Market Cap, P/E Ratio, Dividend Yield, 52 Week High/Low, and Forward PE.
    Use this to answer questions about: 'Is the stock expensive?', current valuation, or dividend info.
    """
    try:
        stock = yf.Ticker(ticker)

        info = stock.info

        finances = "Key Informations:\n" + str(info)

        return finances
    except Exception as e:
        return f"Error fetching key metrics for {ticker}: {str(e)}"

In [32]:
res = get_key_metrics_info.invoke({"ticker": "AAPL"})

In [33]:
print(res)

Key Informations:
{'address1': 'One Apple Park Way', 'city': 'Cupertino', 'state': 'CA', 'zip': '95014', 'country': 'United States', 'phone': '(408) 996-1010', 'website': 'https://www.apple.com', 'industry': 'Consumer Electronics', 'industryKey': 'consumer-electronics', 'industryDisp': 'Consumer Electronics', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple Vision Pro, Apple TV, Apple Watch, Beats products, and HomePod, as well as Apple branded and third-party accessories. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and downloa

In [34]:
@tool
def get_company_summary(ticker: str):
    """
    Useful for understanding what a company does.
    Returns the sector, industry, and a long business summary.
    Use this as the first step when the user asks about a new ticker symbol.
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        sector = info.get('sector', 'Unknown Sector')
        summary = info.get('longBusinessSummary', 'No summary available.')
        # Truncate summary to save tokens if necessary
        return f"Sector: {sector}\nSummary: {summary[:500]}..."
    except Exception as e:
        return f"Error fetching summary for {ticker}: {str(e)}"

In [35]:
res = get_company_summary.invoke({"ticker": "AAPL"})

In [36]:
print(res)

Sector: Technology
Summary: Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple Vision Pro, Apple TV, Apple Watch, Beats products, and HomePod, as well as Apple branded and third-party accessories. It also provides AppleCare support and cloud services; and operates v...


In [ ]:
@tool()      
def get_ticker_news_sentiment(ticker: str):
    """
    Fetches top 10 most relevant news along with sentiment data for a given stock ticker.
    """

    def get_relevance_score(x, ticker):
        for sentiment in x:
            if sentiment['ticker'] == ticker:
                return sentiment['relevance_score']
        return 0
    
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "NEWS_SENTIMENT",
        "tickers": ticker,
        "apikey": os.getenv("alpha_vantage_api_key"),
        "limit": 50
    }
    response = requests.get(url, params=params)
    response = response.json()
    sorted_news = sorted(response['feed'], key=lambda x: get_relevance_score(x['ticker_sentiment'], ticker), reverse=True)

    news = []
    for item in sorted_news[:10]:
        ts = [i for i in item['ticker_sentiment'] if i['ticker'] == ticker][0]
        news.append(f"{item['title']}\nTime of publishing: {item['time_published']}\nAuthors: {",".join(item['authors'])}\nSummary: {item['summary']}\noverall sentiment: {item['overall_sentiment_score']}\noverall sentiment: {item['overall_sentiment_label']}\nticker sentiment: {ts['ticker_sentiment_score']}\nticker sentiment label: {ts['ticker_sentiment_label']}")

    return "\n\n".join(news)

In [64]:
res = get_ticker_news_sentiment.invoke({"ticker": "AAPL"})
print(res)

Apple stock heads into Monday under a valuation spotlight after Raymond James turns neutral on AAPL
Time of publishing: 20260104T215533
Authors: Khadija Saeed
Summary: Raymond James has initiated coverage on Apple (AAPL) with a "market perform" (neutral) rating, citing a rich valuation and a lack of immediate catalysts, despite Apple ending Friday down about 0.3%. Investors are focusing on Apple's upcoming holiday-quarter earnings, iPhone demand, Services growth, and the impact of Apple Intelligence. The stock is about 6% below its 52-week high, with macroeconomic factors like jobs data and the Federal Reserve meeting influencing investor sentiment.
overall sentiment: 0.052992
overall sentiment: Neutral
ticker sentiment: -0.132812
ticker sentiment label: Neutral

Apple Inc. Stock (AAPL) Opinions on iPhone 17 Launch
Time of publishing: 20260104T140738
Authors: Quiver DiscussionTracker
Summary: Social media is abuzz with excitement over the successful iPhone 17 launch and strong early sa

In [138]:
class ticker_state(AgentState):
    ticker: str

In [160]:
@tool()
def set_ticker(ticker: str, runtime: ToolRuntime):
    """Set the ticker symbol for the analysis."""
    return Command[tuple[()]](update={
        "ticker": ticker,
        "messages": [ToolMessage(content=f"Ticker updated to {ticker}", tool_call_id = runtime.tool_call_id)]}, 
        )

@tool()
def get_ticker(runtime: ToolRuntime)-> str:
    """Get the current ticker symbol."""
    return runtime.state.ticker

In [185]:
tools = [get_income_statement, get_balance_sheet, get_cash_flows, get_key_metrics_info, get_company_summary, get_ticker_news_sentiment, get_ticker_symbol, set_ticker, get_ticker]

# The Agent

In [195]:
model = init_chat_model(model = "ministral-3:3b", model_provider = "ollama")

In [196]:
model.invoke("How are you?")

AIMessage(content='Thank you for asking! 😊 I’m here and ready to assist you with anything you need—whether it’s answering questions, helping with planning, or just chatting. How can I make your day a little brighter today?', additional_kwargs={}, response_metadata={'model': 'ministral-3:3b', 'created_at': '2026-01-05T13:28:17.640938Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2661119000, 'load_duration': 163788600, 'prompt_eval_count': 557, 'prompt_eval_duration': 1881837400, 'eval_count': 47, 'eval_duration': 589560900, 'logprobs': None, 'model_name': 'ministral-3:3b', 'model_provider': 'ollama'}, id='lc_run--019b8e58-116f-7752-830f-048b5affb306-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 557, 'output_tokens': 47, 'total_tokens': 604})

In [198]:
system_prompt = """You are a Senior Equity Research Analyst. Your goal is to answer user questions about stock fundamentals using a specific set of financial tools.

### CRITICAL INSTRUCTIONS
1.  **Conciseness is Key:** Your default response style must be **concise, data-backed, and to-the-point**. Do not fluff. Only provide a "Detailed Analysis" if the user explicitly asks for it.
2.  **Tool Economy:** Do NOT call every tool for every request. Use only the specific tools required to answer the user's exact question.
    * *Example:* If asked for "current sentiment," use ONLY `get_ticker_news_sentiment`. Do not fetch the balance sheet.
3.  **Chain of Thought:** You must perform a "thinking" step before answering. In this step, decide which tools are strictly necessary and outline your analysis.
4. **No Hallucinations:** If a tool fails or returns incomplete data, state "Data unavailable for [metric]" instead of making up numbers.

### YOUR TOOLKIT
* `get_company_summary`: For business model, sector, and industry context.
* `get_income_statement`: For revenue, net income, margins, and profitability trends.
* `get_balance_sheet`: For assets, liabilities, debt levels, and liquidity (current ratio).
* `get_cash_flows`: For operating cash flow and Free Cash Flow (FCF) analysis.
* `get_key_metrics_info`: For valuation ratios (P/E, EV/EBITDA) and efficiency (ROE).
* `get_ticker_news_sentiment`: For current market mood, recent headlines, and qualitative risks.

### REASONING PROCESS (Internal)
Before calling tools or answering, you must output a `<thinking>` block:
1.  **Analyze Request:** What is the user specifically asking for?
2.  **Select Tools:** Which tools are *absolutely* needed? (If none, answer from knowledge).
3.  **Plan:** How will I synthesize this data into a concise answer?

### RESPONSE GUIDELINES
* **Standard Mode (Default):** Give a direct answer + 2-3 bullet points of supporting data. Total length < 100 words.
* **Detailed Mode (Only if requested):** Provide an Executive Summary, Financial Breakdown, Valuation, and Risk Assessment.
* **Missing Data:** If a tool fails, state "Data unavailable for [metric]" instead of hallucinating numbers.

### FORMAT
<thinking>
[Your internal reasoning and tool selection logic here]
</thinking>

[Your final concise response here]"""

In [200]:
summarizarion_model = init_chat_model(model="gemma3:1b", model_provider="ollama", temperature=0.7)

agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=system_prompt,
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model = summarizarion_model,
            trigger = ("tokens", 100000),
            keep = ("messages", 10),
            system_prompt = "You are a helpful assistant that summarizes conversation history to save tokens while retaining important information. Summarize previous messages concisely, focusing on key points relevant to ongoing discussion about stock fundamental analysis. Omit any redundant or less important details."
        )
    ]
)

In [201]:
res = agent.invoke(
    {"messages": [
        HumanMessage(content="Is MSFT profitable?")
    ]},
    {"configurable":
     {"thread_id":'1'}}
)

In [202]:
pprint(res)

{'messages': [HumanMessage(content='Is MSFT profitable?', additional_kwargs={}, response_metadata={}, id='d1214499-a284-4144-aa7d-2d9107599789'),
              AIMessage(content="<thinking>\nThe user is asking about Microsoft's profitability. To answer this, I need to fetch Microsoft's income statement data to assess key profitability metrics such as net income, gross margin, and operating margin.\n</thinking>\n\n", additional_kwargs={}, response_metadata={'model': 'ministral-3:3b', 'created_at': '2026-01-05T13:30:39.2262358Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5095820100, 'load_duration': 138168800, 'prompt_eval_count': 1249, 'prompt_eval_duration': 4113539600, 'eval_count': 62, 'eval_duration': 795149000, 'logprobs': None, 'model_name': 'ministral-3:3b', 'model_provider': 'ollama'}, id='lc_run--019b8e5a-3103-77f0-a4c9-134fa281f059-0', tool_calls=[{'name': 'get_income_statement', 'args': {'ticker': 'MSFT'}, 'id': '4f945fed-70db-40b0-af52-8cb3757205d1', 'type': 'to

In [203]:
res = agent.invoke(
    {"messages": [
        HumanMessage(content="What about its valuation?")
    ]},
    {"configurable":
     {"thread_id":'1'}}
)

In [204]:
pprint(res)

{'messages': [HumanMessage(content='Is MSFT profitable?', additional_kwargs={}, response_metadata={}, id='d1214499-a284-4144-aa7d-2d9107599789'),
              AIMessage(content="<thinking>\nThe user is asking about Microsoft's profitability. To answer this, I need to fetch Microsoft's income statement data to assess key profitability metrics such as net income, gross margin, and operating margin.\n</thinking>\n\n", additional_kwargs={}, response_metadata={'model': 'ministral-3:3b', 'created_at': '2026-01-05T13:30:39.2262358Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5095820100, 'load_duration': 138168800, 'prompt_eval_count': 1249, 'prompt_eval_duration': 4113539600, 'eval_count': 62, 'eval_duration': 795149000, 'logprobs': None, 'model_name': 'ministral-3:3b', 'model_provider': 'ollama'}, id='lc_run--019b8e5a-3103-77f0-a4c9-134fa281f059-0', tool_calls=[{'name': 'get_income_statement', 'args': {'ticker': 'MSFT'}, 'id': '4f945fed-70db-40b0-af52-8cb3757205d1', 'type': 'to

In [109]:
pprint(res['messages'][-1].content)

('<thinking>\n'
 '### **Step 1: Business Context**\n'
 'Before diving into financials, let’s clarify Apple’s core business model and '
 'sector positioning.\n'
 '- **Company Summary**: Apple is a global leader in consumer electronics, '
 'software, and services. Its revenue streams include hardware (iPhones, Macs, '
 'iPads), services (App Store, iCloud, Apple Music), and software (iOS, macOS, '
 'iPadOS). Apple operates in the **Technology** sector, specifically within '
 '**Consumer Electronics** and **Software**.\n'
 '- **Key Drivers**: iPhone sales, Services revenue growth (e.g., '
 'subscriptions, cloud services), and innovation (e.g., Apple Intelligence, '
 'Vision Pro). The company is also a major player in **capital markets**, with '
 'a high market cap (~$4 trillion) and significant institutional ownership.\n'
 '\n'
 '---\n'
 '\n'
 '### **Step 2: Financial Health (Fundamentals)**\n'
 'To assess Apple’s financial health, we’ll focus on:\n'
 '1. **Profitability**: Revenue growth

In [86]:
res = agent.invoke(
    {"messages": [
        HumanMessage(content="Sure ")
    ]},
    {"configurable":
     {"thread_id":'1'}}
)

In [87]:
pprint(res['messages'][-1].content)

('Here’s a structured financial summary and analysis of the provided balance '
 'sheet data across four periods (likely years). This analysis focuses on key '
 'financial ratios, trends, and potential insights:\n'
 '\n'
 '---\n'
 '\n'
 '### **1. Key Financial Ratios**\n'
 '#### **Liquidity Ratios**\n'
 '- **Current Ratio** (Current Assets / Current Liabilities):\n'
 '  - Year 1: 1.48 (1.4796M / 0.99M)\n'
 '  - Year 2: 1.53 (1.5299M / 0.99M)\n'
 '  - Year 3: 1.44 (1.4357M / 0.99M)\n'
 '  - Year 4: 1.36 (1.3541M / 0.99M)\n'
 '  **Trend**: Slight decline in liquidity, with Year 4 being the weakest. '
 'Accounts Payable and Cash Equivalents are critical watchpoints.\n'
 '\n'
 '- **Quick Ratio** (Current Assets - Inventory / Current Liabilities):\n'
 '  - Year 1: 0.90 (1.4796M - 0.57M / 0.99M)\n'
 '  - Year 4: 0.81 (1.3541M - 0.49M / 0.99M)\n'
 '  **Implication**: Inventory levels are rising relative to liabilities, '
 'reducing short-term liquidity.\n'
 '\n'
 '#### **Solvency Ratios**\n'
 

# Bringing it all together

In [117]:
class financial_analyst:
    def __init__(self, model, tools, system_prompt, middleware=None):
        """
        Initializes a Financial Analyst agent with the given language model.
        Args:
            model: The language model to be used by the agent.
            tools: The tools available to the agent.
            system_prompt: The system prompt for the agent.
        """
        self.llm = model
        self.tools = tools
        self.system_prompt = system_prompt
        self.checkpointer = InMemorySaver()
        self.middleware = middleware or []
        self.agent = create_agent(
            model=self.llm,
            tools=self.tools,
            system_prompt=self.system_prompt,
            checkpointer=self.checkpointer,
            middleware=self.middleware
        )

    def analyze(self, user_message, thread_id='default_thread'):
        """
        Analyzes the user's message using the Financial Analyst agent.
        Args:
            user_message: The message from the user to be analyzed.
            thread_id: The thread ID for maintaining conversation context.
        """
        response = self.agent.invoke(
            {"messages": [HumanMessage(content=user_message)]},
            {"configurable": {"thread_id": thread_id}}
        )
        return response


In [206]:
modle = init_chat_model(model = "ministral-3:3b", model_provider = "ollama")
fin_agent = financial_analyst(
    model=model,
    tools=tools,
    system_prompt=system_prompt,
    middleware=[
        SummarizationMiddleware(
            model = summarizarion_model,
            trigger = ("tokens", 100000),
            keep = ("messages", 10),
            system_prompt = "You are a helpful assistant that summarizes conversation history to save tokens while retaining important information. Summarize previous messages concisely, focusing on key points relevant to ongoing discussion about stock fundamental analysis. Omit any redundant or less important details."
        )
    ]
)

In [207]:
res = fin_agent.analyze("What is the PE of AAPL", thread_id='1')

In [208]:
res

{'messages': [HumanMessage(content='What is the PE of AAPL', additional_kwargs={}, response_metadata={}, id='a7303e9d-30c3-4014-a03a-df028681196a'),
  AIMessage(content='<thinking>\nThe user is asking for the **Price-to-Earnings (P/E) ratio** of Apple Inc. (AAPL).\nI will use the `get_key_metrics_info` tool to fetch the current P/E ratio and related valuation metrics.\n</thinking>\n\n', additional_kwargs={}, response_metadata={'model': 'ministral-3:3b', 'created_at': '2026-01-05T13:42:44.7839885Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5354266000, 'load_duration': 144678000, 'prompt_eval_count': 1252, 'prompt_eval_duration': 4184978800, 'eval_count': 74, 'eval_duration': 989251700, 'logprobs': None, 'model_name': 'ministral-3:3b', 'model_provider': 'ollama'}, id='lc_run--019b8e65-4242-7442-b2dd-33df0d1db27f-0', tool_calls=[{'name': 'get_key_metrics_info', 'args': {'ticker': 'AAPL'}, 'id': '2d0eb4c5-9cc5-432b-909a-d142ad93caae', 'type': 'tool_call'}], invalid_tool_calls

In [209]:
res = fin_agent.analyze("That of GOOGL", thread_id='1')

In [210]:
res

{'messages': [HumanMessage(content='What is the PE of AAPL', additional_kwargs={}, response_metadata={}, id='a7303e9d-30c3-4014-a03a-df028681196a'),
  AIMessage(content='<thinking>\nThe user is asking for the **Price-to-Earnings (P/E) ratio** of Apple Inc. (AAPL).\nI will use the `get_key_metrics_info` tool to fetch the current P/E ratio and related valuation metrics.\n</thinking>\n\n', additional_kwargs={}, response_metadata={'model': 'ministral-3:3b', 'created_at': '2026-01-05T13:42:44.7839885Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5354266000, 'load_duration': 144678000, 'prompt_eval_count': 1252, 'prompt_eval_duration': 4184978800, 'eval_count': 74, 'eval_duration': 989251700, 'logprobs': None, 'model_name': 'ministral-3:3b', 'model_provider': 'ollama'}, id='lc_run--019b8e65-4242-7442-b2dd-33df0d1db27f-0', tool_calls=[{'name': 'get_key_metrics_info', 'args': {'ticker': 'AAPL'}, 'id': '2d0eb4c5-9cc5-432b-909a-d142ad93caae', 'type': 'tool_call'}], invalid_tool_calls